In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

In [2]:
train = pd.read_csv('train.csv')

In [3]:
def get_sub_stats(sub_area, sub_col, method):
    if(method == 'mode'):
        res = train[(train['sub_area'] == sub_area)][sub_col].mode()
    elif(method == 'median'):
        res = train[(train['sub_area'] == sub_area)][sub_col].median()
    elif(method == 'mean'):
        res = train[(train['sub_area'] == sub_area)][sub_col].mean()
    else:
        return np.NaN

    return res

### Fill trivials

In [4]:
 # fill hospital bed with 0 
train['hospital_beds_raion'] = train['hospital_beds_raion'].fillna(value=0)

In [5]:
# fill with mean for the district
# fill with mode of district as most common rail ID's
train['ID_railroad_station_walk'] = train.groupby(['sub_area'], sort=False)['ID_railroad_station_walk'].apply(lambda x: x.fillna(x.mode()[0])) 
train['metro_min_walk'] = train.groupby(['sub_area'], sort=False)['metro_min_walk'].apply(lambda x: x.fillna(x.mean())) 
train['metro_km_walk'] = train.groupby(['sub_area'], sort=False)['metro_km_walk'].apply(lambda x: x.fillna(x.mean()))
train['railroad_station_walk_km'] = train.groupby(['sub_area'], sort=False)['railroad_station_walk_km'].apply(lambda x: x.fillna(x.mean()))
train['railroad_station_walk_min'] = train.groupby(['sub_area'], sort=False)['railroad_station_walk_min'].apply(lambda x: x.fillna(x.mean()))

In [6]:
# verified that these neighbourhoods have no schools/preschools so we fill null
train['school_quota'] = train['school_quota'].fillna(value=0)
train['preschool_quota'] = train['preschool_quota'].fillna(value=0)

In [7]:
train['material'] = train.groupby(['sub_area'], sort=False)['material'].apply(lambda x: x.fillna(x.mode()[0])) 

### Fill Build_count_mats

In [ ]:
def get_build_index(sub_area, period):
    return ck[c.index(sub_area)][period]

def get_sum(row):
    row_sum = (row['build_count_before_1920']+
        row['build_count_1921-1945'] + 
        row['build_count_1946-1970']+
        row['build_count_1971-1995']+
        row['build_count_after_1995']
    )
    return row_sum

def get_year_built(row):
    periods = [row['build_count_before_1920'],row['build_count_1921-1945'],row['build_count_1946-1970'],row['build_count_1971-1995'],row['build_count_after_1995']]
    max_val = max(periods)
    max_ind = periods.index(max_val)
    if(max_val==0):
        return row['timestamp_year'].astype(float)
    else:
        return mean_build_year[max_ind]

In [8]:
# Material
df['material'] = df.groupby(['sub_area'], sort=False)['material'].apply(lambda x: x.fillna(x.mode()[0])) 

# Build Count
df['build_count_block'] = df['build_count_block'].fillna((np.ceil(df['build_count_block'].mean())))                                  
df['build_count_wood'] = df['build_count_wood'].fillna(value = np.ceil(df['build_count_wood'].mean()))
df['build_count_frame'] = df['build_count_frame'].fillna(value = np.ceil(df['build_count_frame'].mean()))
df['build_count_brick'] = df['build_count_brick'].fillna(value = np.ceil(df['build_count_brick'].mean()))
df['build_count_monolith'] = df['build_count_monolith'].fillna(value = np.ceil(df['build_count_monolith'].mean()))
df['build_count_panel'] = df['build_count_panel'].fillna(value = np.ceil(df['build_count_panel'].mean()))
df['build_count_foam'] = df['build_count_foam'].fillna(value = np.ceil(df['build_count_foam'].mean()))
df['build_count_slag'] = df['build_count_slag'].fillna(value = np.ceil(df['build_count_slag'].mean()))
df['build_count_mix'] = df['build_count_mix'].fillna(value = np.ceil(df['build_count_mix'].mean()))

build_count_sum = (np.ceil(df['build_count_block'].mean())+
   np.ceil(df['build_count_wood'].mean())+
    np.ceil(df['build_count_frame'].mean())+       
    np.ceil(df['build_count_brick'].mean())+
    np.ceil(df['build_count_monolith'].mean())+
    np.ceil(df['build_count_panel'].mean())+
    np.ceil(df['build_count_foam'].mean())+               
    np.ceil(df['build_count_slag'].mean())+
    np.ceil(df['build_count_mix'].mean())                
)

# Build Count Raion
df['raion_build_count_with_material_info'] = df['raion_build_count_with_material_info'].fillna(value=build_count_sum)

c = list(df[df['build_count_1921-1945'].isna()]['sub_area'].unique())

ck = np.ones(shape=(15,5))

for i in range(len(c)):
    build_years = list(df[df['sub_area'] == c[i]]['build_year'].dropna())
    for j in range(len(build_years)):
        if(build_years[j]<=1920):
            ck[i][0] += 1
        elif(1920<build_years[j]<=1945):
            ck[i][1] += 1
        elif(1945<build_years[j]<=1970):
            ck[i][2] += 1
        elif(1970<build_years[j]<=1995):
            ck[i][3] += 1
        else:
            ck[i][4] += 1
            
df['build_count_before_1920'] = df.apply(
    lambda row: get_build_index(row['sub_area'], 0) if np.isnan(row['build_count_before_1920']) else row['build_count_before_1920'],
    axis=1
)

df['build_count_1921-1945'] = df.apply(
    lambda row: get_build_index(row['sub_area'], 1) if np.isnan(row['build_count_1921-1945']) else row['build_count_1921-1945'],
    axis=1
)

df['build_count_1946-1970'] = df.apply(
    lambda row: get_build_index(row['sub_area'], 2) if np.isnan(row['build_count_1946-1970']) else row['build_count_1946-1970'],
    axis=1
)

df['build_count_1971-1995'] = df.apply(
    lambda row: get_build_index(row['sub_area'], 3) if np.isnan(row['build_count_1971-1995']) else row['build_count_1971-1995'],
    axis=1
)

df['build_count_after_1995'] = df.apply(
    lambda row: get_build_index(row['sub_area'], 4) if np.isnan(row['build_count_after_1995']) else row['build_count_after_1995'],
    axis=1
)

df['raion_build_count_with_builddate_info'] = df.apply(
    lambda row: get_sum(row) if np.isnan(row['raion_build_count_with_builddate_info']) else row['raion_build_count_with_builddate_info'],
    axis=1
)

df['build_year'] = df.apply(
    lambda row: np.NaN if ((row['build_year']<1471) | (row['build_year']>2020)) else row['build_year'],
    axis=1
) # set na for bad indexes

mean_build_year = np.ceil(
[
   df[df['build_year']<=1920]['build_year'].mean(),
   df[(df['build_year']>1920) & (df['build_year']<=1945)]['build_year'].mean(),
   df[(df['build_year']>1945) & (df['build_year']<=1970)]['build_year'].mean(),
   df[(df['build_year']>1970) & (df['build_year']<=1995)]['build_year'].mean(),
   df[df['build_year']>1995]['build_year'].mean()
    
]) # get mean build year for each period

df['build_year'] = df.apply(
    lambda row: get_year_built(row) if np.isnan(row['build_year']) else row['build_year'],
    axis=1
)

df['prom_part_5000'] = df.groupby(['sub_area'], sort=False)['prom_part_5000'].apply(lambda x: x.fillna(x.mode()[0]))

cafe_ranges = ['500', '1000', '1500', '2000', '3000', '5000' ]

for cafe_range in cafe_ranges:
    df['cafe_sum_{0}_min_price_avg'.format(cafe_range)] = df.groupby(['sub_area'], sort=False)['cafe_sum_{0}_min_price_avg'.format(cafe_range)].apply(lambda x: x.fillna(x.mean())) 
    df['cafe_sum_{0}_max_price_avg'.format(cafe_range)] = df.groupby(['sub_area'], sort=False)['cafe_sum_{0}_max_price_avg'.format(cafe_range)].apply(lambda x: x.fillna(x.mean())) 
    df['cafe_avg_price_{0}'.format(cafe_range)] = df.groupby(['sub_area'], sort=False)['cafe_avg_price_{0}'.format(cafe_range)].apply(lambda x: x.fillna(x.mean()))
    
    df['cafe_sum_{0}_min_price_avg'.format(cafe_range)] = df['cafe_sum_{0}_min_price_avg'.format(cafe_range)].fillna(value=df['cafe_sum_{0}_min_price_avg'.format(cafe_range)].mean())
    df['cafe_sum_{0}_max_price_avg'.format(cafe_range)] = df['cafe_sum_{0}_max_price_avg'.format(cafe_range)].fillna(value=df['cafe_sum_{0}_max_price_avg'.format(cafe_range)].mean())
    df['cafe_avg_price_{0}'.format(cafe_range)] = df['cafe_avg_price_{0}'.format(cafe_range)].fillna(value=df['cafe_avg_price_{0}'.format(cafe_range)].mean())
    
    

### Fill build_count_years

### Fill build_year

### Fill prom_part / cafe_sum vals

In [22]:
train['prom_part_5000'] = train.groupby(['sub_area'], sort=False)['prom_part_5000'].apply(lambda x: x.fillna(x.mode()[0])) 

In [23]:
cafe_ranges = ['500', '1000', '1500', '2000', '3000', '5000' ]

In [24]:
for cafe_range in cafe_ranges:
    train['cafe_sum_{0}_min_price_avg'.format(cafe_range)] = train.groupby(['sub_area'], sort=False)['cafe_sum_{0}_min_price_avg'.format(cafe_range)].apply(lambda x: x.fillna(x.mean())) 
    train['cafe_sum_{0}_max_price_avg'.format(cafe_range)] = train.groupby(['sub_area'], sort=False)['cafe_sum_{0}_max_price_avg'.format(cafe_range)].apply(lambda x: x.fillna(x.mean())) 
    train['cafe_avg_price_{0}'.format(cafe_range)] = train.groupby(['sub_area'], sort=False)['cafe_avg_price_{0}'.format(cafe_range)].apply(lambda x: x.fillna(x.mean()))
    
    train['cafe_sum_{0}_min_price_avg'.format(cafe_range)] = train['cafe_sum_{0}_min_price_avg'.format(cafe_range)].fillna(value=train['cafe_sum_{0}_min_price_avg'.format(cafe_range)].mean())
    train['cafe_sum_{0}_max_price_avg'.format(cafe_range)] = train['cafe_sum_{0}_max_price_avg'.format(cafe_range)].fillna(value=train['cafe_sum_{0}_max_price_avg'.format(cafe_range)].mean())
    train['cafe_avg_price_{0}'.format(cafe_range)] = train['cafe_avg_price_{0}'.format(cafe_range)].fillna(value=train['cafe_avg_price_{0}'.format(cafe_range)].mean())

### Check Nulls

In [25]:
null_dict = {}
for col in train.columns:
    if( train[col].isnull().sum()>0):
        null_dict[col] = train[col].isnull().sum()
        
{k: v for k, v in sorted(null_dict.items(), key=lambda item: item[1])}

{'floor': 167,
 'life_sq': 6383,
 'max_floor': 9572,
 'num_room': 9572,
 'kitch_sq': 9572,
 'state': 13559}

In [ ]:
df['product_type'] = df.groupby(['sub_area'], sort=False)['product_type'].apply(lambda x: x.fillna(x.mode()[0]))
df['green_part_2000'] = df.groupby(['sub_area'], sort=False)['green_part_2000'].apply(lambda x: x.fillna(x.mode()[0]))